<a href="https://colab.research.google.com/github/jamelof23/ASL2/blob/main/ASL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload dataset

Upload archive.zip to /content/sample_data/

# Unzip the Dataset

In [ ]:
!unzip /content/sample_data/archive.zip -d /content/asl_alphabet

# Organizing and Sorting the data

Create folders for each class in the test directory and move the images into their respective folders. For training data its already sorted.

In [10]:
import shutil
import os

test_dir = '/content/asl_alphabet/asl_alphabet_test/asl_alphabet_test/'

# Define the classes
class_names = ['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I',
               'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
               'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'nothing', 'space']

# Create directories for each class in the test directory
for class_name in class_names:
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# Move each test image into the appropriate class folder
for image_name in os.listdir(test_dir):
    if '_test.jpg' in image_name:
        class_name = image_name.split('_')[0]  # Get the class from the image name
        shutil.move(os.path.join(test_dir, image_name),
                    os.path.join(test_dir, class_name, image_name))

# Load the training and testing datasets

1. Batch Size
Definition: The batch size refers to the number of training examples used in one iteration of training.
Purpose: Instead of updating the model weights after each individual sample, which can be computationally expensive and lead to noisy updates, the model processes a batch of samples and computes the gradient based on that batch. This approach helps to stabilize the training process and can lead to faster convergence.
Common Values: Common batch sizes are powers of 2 (like 32, 64, 128) since they can be more efficient for hardware acceleration (e.g., GPUs).
2. Reproducibility
Definition: Reproducibility refers to the ability to achieve the same results when you run the experiment multiple times under the same conditions.
Purpose: In machine learning, due to the stochastic nature of algorithms (like random initialization of weights or shuffling of data), you might get different results on different runs. Setting a random seed (like seed=123 in the code) ensures that the random processes in your code (e.g., shuffling data or initializing weights) are the same every time you run the model. This way, you can replicate results and debug issues more easily.
3. Shuffle the Dataset
Definition: Shuffling the dataset means randomizing the order of the data samples before they are fed into the model for training.
Purpose: Shuffling is important to ensure that the model does not learn any unintended patterns based on the order of the data (e.g., if all images of one class appear consecutively). Randomizing the input helps the model generalize better and improves performance.

In [ ]:
import tensorflow as tf

test_dir = '/content/asl_alphabet/asl_alphabet_test/asl_alphabet_test/'
train_dir = '/content/asl_alphabet/asl_alphabet_train/asl_alphabet_train/'

# Set image parameters
IMG_HEIGHT, IMG_WIDTH = 200, 200
BATCH_SIZE = 32  # Adjust as needed
SEED = 123  # Set the seed for reproducibility

# Load training data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,  # Shuffle the training data
    seed=SEED  # Set the seed for reproducibility
)

# Load testing data
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,  # Shuffle the training data
    seed=SEED  # Set the seed for reproducibility
)

# Check the class names
class_names = train_ds.class_names
print("Class names:", class_names)


# Finding missing testing image

In [ ]:
import os

# Count test images per class
for class_name in class_names:
    class_path = os.path.join(test_dir, class_name)
    num_images = len(os.listdir(class_path)) if os.path.exists(class_path) else 0
    print(f"{class_name}: {num_images} images")

# Augmenting the missing testing image

Augment the Test Dataset: select the a random image from training dataset to be in the test set

In [ ]:
import os
import random
import shutil

# Define paths
train_dir = '/content/asl_alphabet/asl_alphabet_train/asl_alphabet_train/'
test_dir = '/content/asl_alphabet/asl_alphabet_test/asl_alphabet_test/'

# Specify the class that needs an image
class_name = 'del'  # 'delete' class

# Define the paths for the training and test class directories
train_class_path = os.path.join(train_dir, class_name)
test_class_path = os.path.join(test_dir, class_name)

# Create the test class directory if it doesn't exist
os.makedirs(test_class_path, exist_ok=True)

# Get all images in the training class folder
train_images = [img for img in os.listdir(train_class_path) if img.endswith('.jpg')]

if train_images:
    # Randomly select an image
    random_image = random.choice(train_images)

    # Define source and destination paths with the new name
    src_path = os.path.join(train_class_path, random_image)
    dest_path = os.path.join(test_class_path, f"{class_name}_test.jpg")

    # Move the image from train to test and rename
    shutil.move(src_path, dest_path)
    print(f"Moved {random_image} from {class_name} training set to test set as {class_name}_test.jpg.")
else:
    print(f"No images found in {class_name} training set.")


Build Your Model: If you're ready, you can start building your model to train on the dataset.

In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

# Define directories
train_dir = '/content/asl_alphabet/asl_alphabet_train/asl_alphabet_train/'
output_dir = '/content/asl_alphabet/hand_landmarks/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process images in the training directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)

        # Read the image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process the image to find hands
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Create a list of landmarks
                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

                # Save landmarks as a numpy array or other format
                landmarks_array = np.array(landmarks).flatten()  # Flatten the landmarks

                # Define output path for landmarks
                output_file = os.path.join(output_dir, f"{class_name}_{image_name}.npy")
                np.save(output_file, landmarks_array)
        else:
            print(f"No hands detected in {image_name}")


use MediaPipe’s hands module to detect hand landmarks in your training images.

Processing Images: For each image in your training dataset:
It reads the image and converts it to RGB (MediaPipe expects RGB input).
It processes the image to find hand landmarks.
If hand landmarks are found, they are extracted and saved as a flattened NumPy array.
The output is saved in a specified directory with a unique filename.

Train Your Model
After extracting landmarks from your images, you can use the resulting files (e.g., .npy) as input features for training your model. The training labels can still be derived from your original dataset.

Building a Model
You can now build and train your model using frameworks like TensorFlow or PyTorch. For example:

In [ ]:
import tensorflow as tf

# Load your data (X: landmarks, y: labels)
# Implement your data loading logic here

# Build your model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(landmark_dim,)),  # Adjust landmark_dim
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # num_classes based on your dataset
])

# Compile and train your model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=32)  # Adjust epochs and batch_size as needed
